# How to return structrued output from the prebuilt ReAct agent

## Load LLM

In [1]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="qwen2.5:14b",
    base_url='http://127.0.0.1:11434/v1',
    api_key='ollama',
    temperature=0)

## Define Tools

In [2]:
from typing import Literal
from langchain_core.tools import tool

@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown city")


tools = [get_weather]

# Define Output Structured

In [3]:
from pydantic import BaseModel, Field

class WeatherResponse(BaseModel):
    """Respond to the user in this format"""

    conditions: str = Field(description="Weather conditions")

# Define the graph

In [4]:
from langgraph.prebuilt import create_react_agent

graph = create_react_agent(
    model,
    tools= tools, 
    response_format=WeatherResponse
)

# Usage

In [5]:
inputs = {"messages": [("user", "What's the weather in NYC?")]}
response = graph.invoke(inputs)

In [6]:
response

{'messages': [HumanMessage(content="What's the weather in NYC?", additional_kwargs={}, response_metadata={}, id='62331010-1711-44e0-aa81-9fefcfdb9025'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_vej8jfl8', 'function': {'arguments': '{"city":"nyc"}', 'name': 'get_weather'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 167, 'total_tokens': 188, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen2.5:14b', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-103', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-024ab451-6e6c-4038-94fe-7e18d7947b7c-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'nyc'}, 'id': 'call_vej8jfl8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 167, 'output_tokens': 21, 'total_tokens': 188, 'input_token_details': {}, 'output_token_details': {}}),
  ToolMessage(content='It might be cloudy in ny

In [7]:
response['structured_response']

WeatherResponse(conditions='cloudy')

## Customizing prompt

In [13]:
graph = create_react_agent(
    model,
    tools=tools,
    # specify both the system prompt and the schema for the structured output
    response_format=("Always return capitalized weather conditions", WeatherResponse),
)

inputs = {"messages": [("user", "What's the weather in NYC?")]}
response = graph.invoke(inputs)

In [14]:
response['structured_response']

WeatherResponse(conditions='CLOUDY')